<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Práctica 3 - Enumerativos </h2>
</center>
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>
</div>

En ésta práctica utilizaremos el concepto de ramificación y poda para la resolución del siguiente problema.

La construcción de un artefacto requiere ensamblar $M$ piezas que identiﬁcamos con números entre $1$ y $M$. El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas. Los costes, que son valores positivos, se nos proporcionan en una matriz $C$. La celda $c_{ij}$ es el coste de ensamblar la pieza $i$ cuando ya se han ensamblado $j$ piezas. Deseamos calcular el orden de ensamblado de menor coste total, entendido éste como la suma del coste de ensamblaje de cada una de las piezas. 

<div class="alert alert-info">
<center>
  <h1>Contenido</h1>
  </center><p>
    </p></div>




<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>Ramificación y poda: Problema del ensamblaje</p></h2>
  
  <div class="row">
  <div>
  <p style="text-align: justify; padding-right: 30px">
  Implementa mediante ramificación y poda una solución al problema del ensamblaje con tal de encontrar el listado ordenado de piezas cuyo coste total de ensamblaje sea mínimo.
  </p>
  </div>
    </div></div>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center> 

<p>
<h3>INPUT</h3>
<ul>
<li>C: Matriz de costes de tamaño $M\times M$, donde M es el número de piezas. El elemento $i,j$ de la matriz representa el coste de ensamblaje para la pieza $i$ tras haber colocado $j$ piezas. 
</li>
</ul>
<br>
<h3>OUTPUT</h3>
Lista de números de $1$ a $M$ ordenados de forma que el coste total de ensamblaje sea mínimo. Ej: [5, 3, 2, 4, 1]
</p>

</div>

In [520]:
import heapq
import numpy as np
#init
def branch_bound(C,M):
    """
    Función que obtiene el orden óptimo de las piezas tal que el coste total es mínimo.
    Args:
        - C: matriz de costes M x M
    """
    #matriz donde podremos seleccionar correctamente los items
    matriz=np.empty(shape=(M,M),dtype=bool)
    matriz.fill(True)
    
    cota_sup=cota_superior(C)
    heap=[ ] #cua de prioritats
    coste_acumulado=0 #el coste que acumulamos en cada ramificacion y tenemos el coste de la rama actual
    items= [-1]*M #conjunt d'items resultant
    
    v=[]
    
    columna=0
    
    #inicialitzem la cua
    for i in range(M):
        cost=get_cost(C,M,0,i,v.copy())#minimo fila + coste objeto
        coste_acumulado=C[0][i] #coste de la rama, se acumulará posteriormente
        
        
        it_private=items.copy()
        it_private[i]=0
        matriz_n=matriz.copy()
        matriz_n=fill_matrix(M,i,matriz_n)
        heap.append((cost,coste_acumulado,it_private.index(0),it_private,matriz_n,i))
        coste_acumulado=0


    heapq.heapify(heap)
    while len(heap)!=0:
        
        actual, acc, elemento, its, mat, col = heapq.heappop(heap)
        
        if actual>cota_sup:
            continue

            for i in range(M):
                if mat[][i]: #aqui
                    _cost=get_cost(C,M,elemento,v.copy())
                    acc+=C[elemento][i]
                    n_mat=mat.copy()
                    n_mat=fill_matrix(M,i,n_mat)
                    n_its=its.copy()
                    n_its[i]=elemento
                    heap.append((_cost,acc,elemento,n_its,n_mat,i))
            heapq.heapify(heap)

        
            
    
                    

def get_cost(C,M,obj,col,visited):
    cost=C[obj][col]
    j_temporal=0
    for i in range(obj+1,len(C)):
        less=float('inf')
        for j in range(len(C)):
            if j not in visited:
                if C[i][j]<less and j!=col:
                    less=C[i][j]
                    j_temporal=j
        visited.append(j_temporal)
        cost+=less
    return cost                
              
            
def fill_matrix(M,obj,matrix):
    for i in range(M):
        matrix[i][obj]=False
        matrix[obj][i]=False
        
    return matrix
        
        

#Cogemos el menor de cada fila
def cota_inferior(C):
    inferior=0
    for i in C.keys():
        less=min(C[i].values())
        inferior+=less
    return inferior
#Cogemos los valores de la diagonal
def cota_superior(C):
    superior=0
    for i in C.keys():
        diag=C[i][i]
        superior+=diag
    return superior

In [521]:
import random
from random import randint
random.seed(0)
M = 10 # Número de piezas
# Matriz de costes generada aleatoriamente
C = {}
for i in range(M):
    C[i] = {}
    for j in range(M):
        C[i][j] = randint(1, 25)

    #print(i,C[i])

p={0:{0:11,1:12,2:18,3:40},
   1:{0:14,1:15,2:13,3:22},
   2:{0:11,1:17,2:19,3:23},
   3:{0:17,1:14,2:20,3:28}}
xx={0:{0:11,1:14,2:11,3:17},
   1:{0:12,1:15,2:17,3:14},
   2:{0:18,1:13,2:19,3:20},
   3:{0:40,1:22,2:23,3:28}}
n={0:{0:5,1:6,2:2,3:11},
  1:{0:22,1:1,2:4,3:12},
  2:{0:8,1:7,2:12,3:13},
  3:{0:14,1:16,2:20,3:3}}
N=4
solucion = branch_bound(n,N)
print(solucion)

[(21, 5, 0, [0, -1, -1, -1], array([[False, False, False, False],
       [False,  True,  True,  True],
       [False,  True,  True,  True],
       [False,  True,  True,  True]], dtype=bool), 0), (21, 6, 1, [-1, 0, -1, -1], array([[ True, False,  True,  True],
       [False, False, False, False],
       [ True, False,  True,  True],
       [ True, False,  True,  True]], dtype=bool), 1), (14, 2, 2, [-1, -1, 0, -1], array([[ True,  True, False,  True],
       [ True,  True, False,  True],
       [False, False, False, False],
       [ True,  True, False,  True]], dtype=bool), 2), (40, 11, 3, [-1, -1, -1, 0], array([[ True,  True,  True, False],
       [ True,  True,  True, False],
       [ True,  True,  True, False],
       [False, False, False, False]], dtype=bool), 3)]
21
fv 3
None


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios </h3></p> </center> 

</div>

### En esta sección debéis responder a las preguntas siguientes:
#### 1. Explica cómo has realizado tu implementación, enfatizando en la heurística empleada.
#### 2. Análisis de complejidad de la implementación

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>

<p>
La entrega tiene que consistir en un solo fichero **.ipynb** con el nombre: *Apellido1_Apellido2*_*Nombre*_ASS3.ipyn
</p>
</div>